In [1]:
import os
import glob

def remove_files_with_pattern(folder_path, pattern):
    # Create the full pattern for glob
    full_pattern = os.path.join(folder_path, pattern)
    
    # Use glob to find files matching the pattern
    files_to_remove = glob.glob(full_pattern)
    
    # Remove each file
    for file_path in files_to_remove:
        try:
            os.remove(file_path)
            print(f"Removed: {file_path}")
        except Exception as e:
            print(f"Error removing {file_path}: {e}")

# Example usage
folder_path = 'blob'  # Replace with your folder path
pattern = '*'  # Replace with your desired pattern (e.g., '*.jpg', '*.log', etc.)
remove_files_with_pattern(folder_path, pattern)


Removed: blob/imagen_45.png
Removed: blob/imagen_184.png
Removed: blob/imagen_12.png
Removed: blob/imagen_221.png
Removed: blob/imagen_33.png
Removed: blob/imagen_224.png
Removed: blob/imagen_24.png
Removed: blob/imagen_168.png
Removed: blob/imagen_20.png
Removed: blob/imagen_108.png
Removed: blob/imagen_133.png
Removed: blob/imagen_19.png
Removed: blob/imagen_14.png
Removed: blob/imagen_74.png
Removed: blob/imagen_120.png
Removed: blob/imagen_83.png
Removed: blob/imagen_98.png
Removed: blob/imagen_256.png
Removed: blob/imagen_255.png
Removed: blob/imagen_42.png
Removed: blob/imagen_71.png
Removed: blob/imagen_208.png
Removed: blob/imagen_186.png
Removed: blob/imagen_25.png
Removed: blob/imagen_76.png
Removed: blob/imagen_57.png
Removed: blob/imagen_148.png
Removed: blob/imagen_176.png
Removed: blob/imagen_55.png
Removed: blob/imagen_188.png
Removed: blob/imagen_53.png
Removed: blob/imagen_211.png
Removed: blob/imagen_156.png
Removed: blob/imagen_143.png
Removed: blob/imagen_117.png
Re

In [2]:
import pandas as pd
import matplotlib.pyplot as plt

# Cargar datos desde el CSV
df = pd.read_csv('../CPM/output.csv')
df.columns = df.columns.str.strip()
print(df.head())


         t         mx         my    mr       p0x        p0y    p0r        p1x  \
0  0.00000  99.700000  35.000000  0.15  2.798080  48.136284  0.150  33.316529   
1  0.01875  99.689071  35.000000  0.17  2.804577  48.135403  0.162  33.323075   
2  0.03750  99.659701  35.000590  0.21  2.822039  48.133035  0.186  33.340669   
3  0.05625  99.604899  35.001902  0.27  2.854625  48.128616  0.222  33.373502   
4  0.07500  99.529933  35.004179  0.32  2.906229  48.121615  0.270  33.425498   

         p1y    p1r  ...   p11r       p12x       p12y   p12r       p13x  \
0  38.702802  0.150  ...  0.150  85.933212  11.407552  0.150  25.431226   
1  38.702437  0.162  ...  0.162  85.936516  11.413214  0.162  25.437618   
2  38.701456  0.186  ...  0.186  85.945276  11.428504  0.186  25.454797   
3  38.699624  0.222  ...  0.222  85.961579  11.457062  0.222  25.486855   
4  38.696723  0.270  ...  0.270  85.987270  11.502361  0.270  25.537621   

        p13y   p13r       p14x       p14y   p14r  
0  18.03966

In [3]:
# Recorrer cada fila del DataFrame
min_r = 0.15
for index, row in df.iterrows():
    fig, ax = plt.subplots()
    # Dibujar el círculo azul (mx, my, mr)
    circle = plt.Circle((row['mx'], row['my']), min_r, color='blue')
    circle_dashed = plt.Circle((row['mx'], row['my']), row['mr'], color='blue', fill=False, linestyle='--')
    ax.add_patch(circle)

    # Dibujar los puntos rojos (p0x, p0y, ..., p14x, p14y) con su radio correspondiente (p0r, p1r, ..., p14r)
    for i in range(15):
        circle = plt.Circle((row[f'p{i}x'], row[f'p{i}y']), min_r, color='red')
        circle_dashed = plt.Circle((row[f'p{i}x'], row[f'p{i}y']), row[f'p{i}r'], color='blue', fill=False, linestyle='--')# Radio ajustado con pnr
        ax.add_patch(circle)

    # Configuración de ejes
    ax.set_aspect('equal')
    ax.set_xlim(0, 100)  # Limite en el eje X de 0 a 100
    ax.set_ylim(0, 70)   # Limite en el eje Y de 0 a 70
    # Guardar la imagen con un nombre único (por ejemplo usando el índice)
    mx = row['mx']
    my = row['my']
    plt.title(f'maradona: ({mx}, {my})')
    plt.savefig(f'blob/imagen_{index}.png')
    
    plt.close(fig)


In [4]:
import glob
import os
import imageio.v2 as imageio
from tqdm import tqdm

def create_video(input_pattern, output_file, fps=30):
    # Create a list of images
    images = sorted(glob.glob(input_pattern), key=lambda x: float(x.split('_')[-1][:-4]))
    
    # Check if images were found
    if not images:
        print(f"No images found matching the pattern: {input_pattern}")
        return False

    try:
        # Get the first image to determine dimensions
        first_image = imageio.imread(images[0])
        height, width = first_image.shape[:2]

        # Create a writer object
        writer = imageio.get_writer(output_file, fps=fps, macro_block_size=None)

        # Read and write images
        for image_path in tqdm(images, desc="Processing images"):
            image = imageio.imread(image_path)
            writer.append_data(image)

        # Close the writer
        writer.close()
        
        print(f"Video created successfully: {output_file}")
        return True
    except Exception as e:
        print(f"An error occurred while creating the video: {e}")
        return False

# Usage
input_pattern = "blob/imagen_*.png"
output_file = 'animacion.mp4'
success = create_video(input_pattern, output_file)


Processing images: 100%|██████████| 407/407 [00:02<00:00, 138.09it/s]


Video created successfully: animacion.mp4


In [5]:
from IPython.display import Video
url = 'animacion.mp4'
Video(url)